In [4]:
import os
import sys
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import ast
from sklearn.ensemble import RandomForestRegressor
sys.path.insert(0, os.path.abspath("../../" + 'src/scraping'))
import metadata as meta
sys.path.insert(0, os.path.abspath("../../" + "src/modeling"))
import basic_stats as basic
import time

In [5]:
big_df = pd.read_csv('../../data/local/fortnite/video_data/merged_df.csv').drop("Unnamed: 0",axis=1)

In [6]:
all_meta = pd.read_csv("../../data/local/fortnite/video_data/fortnite_master_metadata.csv")

In [7]:
save_dir = "../../data/local/fortnite/thumbnails/"
chan_ids = all_meta['channelId'].value_counts().index
def get_all_thumbnails(save_dir, chan_ids, big_df):
    for chan_id in chan_ids:
        chan_df = all_meta[all_meta['channelId'] == chan_id]
        chan_vids = meta.download_df_thumbs(chan_df,save_dir,'medium')
    chan_basic_df = basic.basic_image_stats(save_dir, "../../data/local/fortnite/video_data/","bleh")
    full_chan = all_meta.merge(chan_basic_df,on='videoId')
    big_df = pd.concat([big_df,full_chan],sort=True)
    big_df = big_df.drop_duplicates(subset='videoId')
    return big_df

In [ ]:
comb_df = big_df.drop_duplicates(subset='videoId')

In [ ]:
full_sypher = all_meta.merge(sypher_vids,on='videoId')

In [ ]:
# (big_df['viewCount'] / 1000).describe()

In [ ]:
sorted_df = big_df.sort_values(by='position',axis=0).reset_index()
# pos = sorted_df['position']
# views = sorted_df['viewCount']

In [ ]:
# sorted_df['emotions'].value_counts()

In [ ]:
# sorted_df['num_rgb']

In [ ]:
# sorted_df[emotion == True]

In [ ]:
emotions = ['angry', 'happy', 'sad', 'fear', 'neutral', 'surprise']
for targ in emotions:
# targ = emotions[1]
    emotion = sorted_df['emotions'].apply(lambda x: targ in x if type(x) == str else x)
    print("Emotion:",targ)
    display(sorted_df[emotion == True]['z_views'].describe())
    plt.scatter(emotion,sorted_df['z_views'])
    plt.show()

In [ ]:
emotion = sorted_df['emotions'].apply(lambda x: len(ast.literal_eval(x)) if type(x) == str else 0)
# print("Emotion:",targ)
display(emotion.describe())
plt.scatter(emotion,sorted_df['z_views'])
# plt.show()

In [ ]:
sorted_df.columns

In [ ]:
basic_img_feats = ['num_rgb','mean_hue','mean_saturation','mean_brightness','contrast','edge_score']
meta_feats = ['z_views']
def get_corr(img_feats, meta_feats,df,sign):
    for col in meta_feats:
        for col2 in img_feats:
        #     col = 'z_views'
        #     col2 = 'contrast'
            step1 = df[df[col].notnull()]
            filtered = step1[step1[col2].notnull()]
            corr = np.corrcoef(filtered[col],filtered[col2])[0][1]
            if sign == "-":
                if corr < 0:
                    print(col, col2, corr)
                    print('---------------')
            else:
                if corr > 0:
                    print(col, col2, corr)
                    print('---------------')
# get_corr(basic_img_feats, meta_feats, big_df)

In [ ]:
sorted_df['z_views'].describe()[['count','mean','50%']]

In [ ]:
def get_range(col_name, df):
    stats = df[col_name].describe()
    col_range = np.arange(stats['25%'],stats['max'],stats['std']/3)
    return col_range

In [ ]:
def tune_params(df, c1, c2):
    baseline_stats = df['z_views'].describe()
    base_mean = baseline_stats['mean']
    base_median = baseline_stats['50%']
    print("baseline:     ",
          "count:",baseline_stats['count'],
          "mean:",baseline_stats['mean'].round(3),
          "median:",baseline_stats['50%'].round(3))
    # print('---------------------------------------')
    #['baseline',0,baseline_stats['count'],baseline_stats['mean'],baseline_stats['50%']]
    rel_stats = []
    c1_range = get_range(c1,df)
    c2_range = get_range(c2,df)
    for c1_cutoff in c1_range:
        for c2_cutoff in c2_range:
            filtered = df.apply(lambda x: x[c1] > c1_cutoff and x[c2] > c2_cutoff,axis=1)
            stats = df[filtered]['z_views'].describe()
            cur_count = stats['count']
            cur_mean = stats['mean']
            cur_median = stats['50%']
            cur_stats_list = [c1,c1_cutoff,c2,c2_cutoff,cur_count, cur_mean, cur_median]
            rel_stats.append(cur_stats_list)
    res_df = pd.DataFrame(rel_stats, columns=['col1','col1cutoff','col2','col2cutoff','count','mean','median'])
    not_null_df = res_df[res_df['mean'].notnull()]
    better_df = not_null_df[not_null_df.apply(
        lambda x: x['mean'] > base_mean and x['median'] > base_median,axis=1)]
    return better_df

In [ ]:
c1 = 'num_rgb'
c2 = 'mean_saturation'
# comb_df = pd.concat([big_df,full_sypher],sort=True).drop_duplicates(subset='videoId')

In [ ]:
print(comb_df[c1].describe())
print(comb_df[c2].describe())

In [ ]:
get_corr(basic_img_feats, meta_feats, comb_df, '-')

In [ ]:
get_corr(basic_img_feats, meta_feats, comb_df, '+')

In [ ]:
tuned_df = tune_params(comb_df)
tuned_df['mm_sum'] = tuned_df['mean'] + tuned_df['median']

In [ ]:
top_df = tuned_df.sort_values(by='mm_sum',ascending=False)

In [ ]:
rgb_cutoff = top_df['col1cutoff'].describe()['mean']
sat_cutoff = top_df['col2cutoff'].describe()['mean']

In [ ]:
comb_df['z_views'].describe()

In [ ]:
top_cut = comb_df[comb_df.apply(
    lambda x: x['num_rgb'] > rgb_cutoff and x['mean_saturation'] > sat_cutoff,axis=1)]
top_cut['z_views'].describe()

In [ ]:
top_200 = pd.read_csv('../../data/local/fortnite/video_data/merged_df.csv').drop("Unnamed: 0",axis=1)
full_views = top_200[top_200['z_views'].notnull()]

In [ ]:
basic_img_feats

In [ ]:
rf = RandomForestRegressor()
rf.fit(full_views[basic_img_feats],full_views[['position']])
rf.feature_importances_

In [ ]:
# baseline_stats = sorted_df['z_views'].describe()
# print("baseline:     ",
#       "count:",baseline_stats['count'],
#       "mean:",baseline_stats['mean'].round(3),
#       "median:",baseline_stats['50%'].round(3))
# # print('---------------------------------------')
# #['baseline',0,baseline_stats['count'],baseline_stats['mean'],baseline_stats['50%']]
# rel_stats = []
# c1 = 'num_rgb'
# c1_range = get_range(c1,sorted_df)
# c2 = 'mean_saturation'
# c2_range = get_range(c2, sorted_df)
# for c1_cutoff in c1_range:
#     for c2_cutoff in c2_range:
#         filtered = sorted_df.apply(lambda x: x[c1] > c1_cutoff and x[c2] > c2_cutoff,axis=1)
#         stats = sorted_df[filtered]['z_views'].describe()
#         cur_count = stats['count']
#         cur_mean = stats['mean']
#         cur_median = stats['50%']
#         cur_stats_list = [c1,c1_cutoff,c2,c2_cutoff,cur_count, cur_mean, cur_median]
#         rel_stats.append(cur_stats_list)
# res_df = pd.DataFrame(rel_stats, columns=['col1','col1cutoff','col2','col2cutoff','count','mean','median'])

In [ ]:
not_null_df = res_df[res_df['mean'].notnull()]
not_null_df[not_null_df.apply(lambda x: x['mean'] > .5 and x['median']> .2,axis=1)]

In [ ]:
c1 = 'mean_saturation'
c1_stats = sorted_df[c1].describe()
c1_range = np.arange(c1_stats['25%'],c1_stats['75%'],c1_stats['std']/3)
c1_range

In [ ]:
c1_stats

In [ ]:
sorted_df['z_views'].describe()

In [ ]:
col = 'contrast'
print(sorted_df[col].describe())
plt.scatter(sorted_df['contrast'],sorted_df['z_views'])

In [ ]:
sorted_df['contrast'].describe()

In [ ]:
sorted_df.columns

In [ ]:
sys.path.insert(0, os.path.abspath("../../" + 'src/scraping'))
import metadata as meta
sys.path.insert(0, os.path.abspath("../../" + "src/modeling"))
import basic_stats as basic

In [ ]:
df = pd.read_csv("../../data/test/fortnite/video_data/fortnite_full_metadata.csv")
basic_stats_df = pd.read_csv("../../data/test/fortnite/video_data/basic_stats_04_27_20.csv")

In [ ]:
basic_stats_df.set_index("videoId")
df.set_index("videoId")
df = df[df['channelId'].notnull()]

In [ ]:
# for vid_id in basic_stats_df['videoId'].values:
#     if vid_id not in df['videoId'].values:
#         print(vid_id)

In [ ]:
merged_df = df.merge(basic_stats_df,how="left",on="videoId")

In [ ]:
merged_df.columns

In [ ]:
datetime.now().strftime("_%m_%d_%y")

In [ ]:
s = str({"cmd":"LOG","devmac":"CC:50:E3:B0:21:8C","gid":"05", "BLEInfo":{"beacons": [{"mac":"4e:8c:c8:e5:1b:e8","rssi":"-79","msgtype":"iBeacon"},{"mac":"48:ee:a2:7f:38:76","rssi":"-42","msgtype":"iBeacon"},{"mac":"4e:8c:c8:e5:1b:e8","rssi":"-79","msgtype":"iBeacon"},{"mac":"48:ee:a2:7f:38:76","rssi":"-42","msgtype":"iBeacon"},{"mac":"4e:8c:c8:e5:1b:e8","rssi":"-77","msgtype":"iBeacon"},{"mac":"4e:8c:c8:e5:1b:e8","rssi":"-79","msgtype":"iBeacon"}]}})

In [ ]:
s = '{\r\n  "cmd": "GROUPS"\r\n}'
s